# Process the Summary Files

This script will process the *_summary.csv files.  The will iterate through all of the files and generate a single file for a specific growth phase and year.

In [ ]:
#Import the required libraries
import sys
import os
import datetime
import numpy as np
import pandas as pd

In [ ]:
sourcedir = "/OSM/CBR/AG_WHEATTEMP/work/output"

In [ ]:
filelist_df = pd.DataFrame(columns=['filename'])
filelist_df.filename = sorted(sourcedir+'/'+f for f in os.listdir(sourcedir) if f.endswith('_summary.csv'))
print(filelist_df.head())


In [ ]:
filelist_df.to_csv(outfilelist, header=False, index=False)

In [ ]:
filelist_df = pd.read_csv(outfilelist, header=None)
filelist_df.columns=['filename']
filelist_df.head()

In [ ]:
def process_summary_file(filelist, filter_phase, filter_year):
    '''
    processes an individual file and extracts information based on the phase and year that
    is passed in
    '''

    outfilelist = sourcedir + "/" + filelist
    filelist_df = pd.read_csv(outfilelist, header=None)
    filelist_df.columns=['filename']    
    
    for filename in filelist_df.filename:
        #print(" ...file: ", filename)
    
        dfData = pd.read_csv(filename)
        dfData = dfData[(dfData['phases'] == filter_phase)]
        dfData['sowingdate'] = pd.to_datetime(dfData['sowingdate'], format="%Y-%m-%d")
        dfData['year'] = dfData['sowingdate'].dt.year
        dfData['sowdate'] = dfData['sowingdate'].dt.strftime("%d-%B")
        dfData = dfData[(dfData['year'] == filter_year)]

        cols = ['SimID', 'variety', 'long', 'lat', 'sowdate', 'phases', 'dayCount', 
                'maxTemp', 'avgTemp', 'days>=30', 'days>=32']
        dfData = dfData[cols]

        dfData.rename(columns={'days>=30': 'daysGTE30', 'days>=32': 'daysGTE32'}, inplace=True)

        outfile = sourcedir + "/" + filter_phase + "_" + str(filter_year) + ".csv"

        if not os.path.isfile(outfile):
            dfData.to_csv(outfile, header=True, encoding='utf-8', index=False)
        else:
            dfData.to_csv(outfile, header=False, mode='a', encoding='utf-8', index=False)
        
    

In [ ]:
# set some defaults to work with
filelist = "filelist.txt"
filter_phase = "07_GrainFilling"
filter_year = 1957


print("processing started at ", datetime.datetime.now())
process_summary_files(filelist, filter_phase, filter_year)
print("Processs completed", datetime.datetime.now())